In [1]:
import asyncio
from pathlib import Path  # Используем pathlib для удобной и кросс-платформенной работы с путями
import uuid
from typing import Optional, Coroutine, Any

from sensory_data_client import DataClient, get_settings, DataClientConfig, PostgresConfig, MinioConfig, create_data_client
from sensory_data_client.models.document import DocumentCreate, DocumentMetadata
from sensory_data_client.models import GroupCreate

from pydantic import BaseModel, Field
cfg = DataClientConfig(postgres = PostgresConfig(user = "postgres",
                                                           password = "postgres",
                                                           host = "10.10.0.70",
                                                           port=5422), 
                                            minio = MinioConfig(endpoint="10.10.0.70:9008",  bucket = "documents"))
client = create_data_client(cfg)
await client.check_connections()

[sensory-data-client] Database triggers registered for DocumentORM.
endpoint='10.10.0.70:9008' accesskey='minioadmin' secretkey='minioadmin' bucket='documents' secure=False


{'postgres': 'ok', 'minio': 'ok'}

In [2]:

lines  = await client.list_doclines()
objects = await client.list_stor(prefix="pdf/")

sys:1: SAWarning: relationship 'UserGroupMembershipORM.user' will copy column users.id to column user_group_membership.user_id, which conflicts with relationship(s): 'GroupORM.users' (copies users.id to user_group_membership.user_id), 'UserORM.groups' (copies users.id to user_group_membership.user_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="groups,users"' to the 'UserGroupMembershipORM.user' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
sys:1: SAWarning:

ValidationError: 2 validation errors for Line
line_no
  Field required [type=missing, input_value={'doc_id': UUID('19edc8d2...и ПАК «Neuroair»'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
block_type
  Field required [type=missing, input_value={'doc_id': UUID('19edc8d2...и ПАК «Neuroair»'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [3]:
email = await client.get_user_by_email("fox@sensorylab.ru")
email


In [2]:
email = await client.create_user(email="fox@sensorylab.ru", plain_password="1qa2ws#ED")


(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\passlib\handlers\bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
AttributeError: module 'bcrypt' has no attribute '__about__'


In [5]:

group = await client.create_group(GroupCreate(name="Первые пташки", description="Ну пацаны нормальные"))
group


ValidationError: 1 validation error for GroupInDB
edited_at
  Field required [type=missing, input_value=<sensory_data_client.db.u...t at 0x0000028A10334B50>, input_type=GroupORM]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [9]:


file_path_str = r"X:\Downloads\Сенсори_ГПБ_КЕЙСЫ_11.04.2025 final.pdf"
file_path = Path(file_path_str)

# Проверка, что файл существует, перед тем как его читать
if not file_path.is_file():
    print(f"Ошибка: Файл не найден по пути {file_path}")

print(f"Подготовка к загрузке файла: {file_path}")

# Шаг 2: Читаем файл в байты
# .read_bytes() - простой способ прочитать весь файл в память
try:
    file_content: bytes = file_path.read_bytes()
except Exception as e:
    print(f"Не удалось прочитать файл: {e}")
    

# Шаг 3: Создаем объект метаданных DocumentCreate
# Извлекаем информацию из пути и добавляем остальную
document_meta = DocumentCreate(
    user_document_id=str(uuid.uuid4()), # Генерируем уникальный ID для этого документа
    name=file_path.name,                # 'X5Group x SensoryLAB 07.02.2024.pptx.pdf'
    owner_id=email.id,             # ID пользователя, который загружает файл
    access_group_id=uuid.UUID("7ebc8f2f498b418a92b2e6dbf5c5cba7"),         # Опциональная группа доступа
    metadata=DocumentMetadata()
)


Подготовка к загрузке файла: X:\Downloads\Сенсори_ГПБ_КЕЙСЫ_11.04.2025 final.pdf


In [7]:
uuid.UUID("8f088d88fd3b4c86af382196b6cab3cc")

UUID('8f088d88-fd3b-4c86-af38-2196b6cab3cc')

In [10]:

# Шаг 4: Вызываем асинхронный метод с `await`
try:
    # Ваш метод `upload_file` скорее всего захочет получить чистое имя файла,
    # а не весь путь. file_path.name идеально для этого подходит.
    result_document = await client.upload_file(
        file_name=file_path.name,
        content=file_content,
        meta=document_meta
    )
    
    print("\n✅ Успешно загружен документ!")
    print(f"   ID в базе: {result_document.id}")
    print(f"   Путь в хранилище: {result_document}")

except Exception as e:
    print(f"❌ Произошла ошибка во время загрузки: {e}")



✅ Успешно загружен документ!
   ID в базе: 0608b93e-9324-4154-9e8f-cd11316c25fb
   Путь в хранилище: user_document_id='d338d03f-e892-4400-81e0-f27e1ba63aa5' name='Сенсори_ГПБ_КЕЙСЫ_11.04.2025 final.pdf' owner_id=UUID('2988de0b-105c-4afc-b7b9-88dec435a9ba') access_group_id=UUID('7ebc8f2f-498b-418a-92b2-e6dbf5c5cba7') metadata=DocumentMetadata(processing_status=None, image_object_paths=None, extra=None) id=UUID('0608b93e-9324-4154-9e8f-cd11316c25fb') created=datetime.datetime(2025, 8, 16, 19, 27, 47, 44552, tzinfo=datetime.timezone.utc) edited=datetime.datetime(2025, 8, 16, 19, 27, 47, 44552, tzinfo=datetime.timezone.utc) is_sync_enabled=True extension='pdf' content_hash='58309a70ab61b42e8935bae1bbe21e64003c3f44ac6ec759fca052f23ce3c6ac' object_path='pdf/0608b93e932441549e8fcd11316c25fb/raw/Сенсори_ГПБ_КЕЙСЫ_11.04.2025 final.pdf'


In [53]:
import asyncio
import typer
import logging
import sys
if sys.platform == "win32":
    # Принудительно устанавливаем политику, которая использует SelectorEventLoop.
    # Это решает проблему с ProactorEventLoop по умолчанию в Windows.
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    
from sensory_data_client import DataClient, get_settings, DataClientConfig, PostgresConfig, MinioConfig, create_data_client
from sensory_data_client.models.document import DocumentCreate, DocumentMetadata

from sensory_data_client.config import get_settings
from sensory_data_client import create_data_client
from sensory_data_client.exceptions import DataClientError
from sensory_data_client.utils.cli_utils import get_rich_console
settings = get_settings()
# ИМПОРТИРУЕМ Base из ORM и create_async_engine
from sensory_data_client.db.base import Base
from sqlalchemy.ext.asyncio import create_async_engine


app = typer.Typer(help="CLI for sensory-data-client management.")
logger = logging.getLogger(__name__)
console = get_rich_console()
cfg = DataClientConfig(postgres = PostgresConfig(user = "postgres",
                                                           password = "postgres",
                                                           host = "10.10.0.70",
                                                           port=5422), 
                                            minio = MinioConfig(endpoint="10.10.0.70:9008",  bucket = "documents"))

engine = create_async_engine(cfg.postgres.get_pg_dsn())
async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.create_all)
await engine.dispose()


In [55]:

# Шаг 4: Вызываем асинхронный метод с `await`
try:
    # Ваш метод `upload_file` скорее всего захочет получить чистое имя файла,
    # а не весь путь. file_path.name идеально для этого подходит.
    result_document = await client.upload_file(
        file_name=file_path.name,
        content=file_content,
        meta=document_meta
    )
    
    print("\n✅ Успешно загружен документ!")
    print(f"   ID в базе: {result_document.id}")
    print(f"   Путь в хранилище: {result_document}")

except Exception as e:
    print(f"❌ Произошла ошибка во время загрузки: {e}")


❌ Произошла ошибка во время загрузки: Failed to save document metadata: (sqlalchemy.dialects.postgresql.asyncpg.IntegrityError) <class 'asyncpg.exceptions.UniqueViolationError'>: duplicate key value violates unique constraint "uq_documents_owner_userdoc"
DETAIL:  Key (owner_id, user_document_id)=(0c8608a8-6488-442e-be2c-1a431a0a3c04, 8da97dc0-0dc1-4155-afca-a613483fa676) already exists.
[SQL: INSERT INTO documents (id, user_document_id, stored_file_id, name, owner_id, access_group_id, metadata) VALUES ($1::UUID, $2::VARCHAR, $3::INTEGER, $4::VARCHAR, $5::UUID, $6::UUID, $7::JSONB) RETURNING documents.created, documents.edited]
[parameters: (UUID('ff949cd2-9d15-416b-980f-d68c39aa722e'), '8da97dc0-0dc1-4155-afca-a613483fa676', 1, 'ai-hr-2024.pdf', UUID('0c8608a8-6488-442e-be2c-1a431a0a3c04'), UUID('8f088d88-fd3b-4c86-af38-2196b6cab3cc'), '{"processing_status": null, "image_object_paths": null, "extra": null}')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [52]:
doc_meta = await client.metarepo.get("8a95427a-18d7-4d50-8ec6-1b631ae70de6")

ValidationError: 2 validation errors for DocumentInDB
owner_id
  Input should be a valid string [type=string_type, input_value=UUID('8f16fc0c-e4d0-42ef-89a2-3aa787801f56'), input_type=UUID]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
access_group_id
  Input should be a valid string [type=string_type, input_value=UUID('8f088d88-fd3b-4c86-af38-2196b6cab3cc'), input_type=UUID]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

In [ ]:
cfg = DataClientConfig(postgres = PostgresConfig(user = "postgres",
                                                           password = "postgres",
                                                           host = "10.10.0.70",
                                                           port=5422), 
                                            minio = MinioConfig(endpoint="10.10.0.70:9008",  bucket = "documents"))
